# Tarea 1
Dado el corpus proporcionado, obtener la curva de Zipf de las palabras en
escala logaritmica.
Pasos:

1. Limpiar el corpus: eliminar signos de puntuacion, de interrogacion, admiracion y elementos no lexicos.

2. Aplicar un algoritmo de Stemming a los tokens limpios.
3. Obtener las frecuencias de los tipos en el corpus.
4. Ordenar por el rango estadistico de mayor a menor.
5. Graficar el diagrama de dispersion rango-frecuencia en escala logarıtmica.

In [76]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos.

In [139]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"\CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        
def identifyWords(text):
    """
    Esta función divide las palabras por espacio y limpia las palabras de los caracteres especiales
    """
    text.replace(',',' ');
    palabras = text.split()
    base=[]
    for val in palabras:
        base.append(val.strip('(\\{})^?=,_-.[]'))
    return base

In [16]:
texts = loadAllCorpus()
text = loadCorpus('acustica (2).txt')
print(text)

Energía sonora
La energía sonora (o energía acústica) es la energía que transmiten o transportan las ondas sonoras. Procede de la energía vibracional del foco sonoro y se propaga a las partículas del medio que atraviesan en forma de energía cinética (movimiento de las partículas), y de energía potencial (cambios de presión producidos en dicho medio,o presión sonora). Al irse propagando el sonido a través del medio, la energía se transmite a la velocidad de la onda, pero una parte de la energía sonora se disipa en forma de energía térmica.1 La energía acústica suele tener valores absolutos bajos, y su unidad de medida es el joule (J). Aunque puede calcularse a partir de otras magnitudes como la intensidad sonora, también se pueden calcular otras magnitudes relacionadas,como la densidad o el flujo de energía acústica.
A partir de la definición de intensidad acústica, se puede calcular la energía acústica que atraviesa una superficie A:
E_{{son}}=\int _{A}\left[\int _{0}^{{\Delta t}}I_{{s

In [140]:
base = identifyWords(text)
print(base[224])

Referencias
